# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,1.42,76,95,10.54,TF,1723139127
1,1,waitangi,-43.9535,-176.5597,11.04,79,68,6.64,NZ,1723139128
2,2,grytviken,-54.2811,-36.5092,-12.31,87,87,7.72,GS,1723139129
3,3,jutai,-5.1833,-68.9000,38.36,27,1,0.50,BR,1723139131
4,4,flin flon,54.7682,-101.8650,17.91,52,75,3.09,CA,1723139029


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

locations = city_data_df[['Lat', 'Lng']]
Humidity = city_data_df['Humidity']
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size=Humidity / 10, color = 'Red', alpha=0.5, titles='ESRI', hover_cols=['City', 'Humidity'])
map_plot

:Points   [Lng,Lat]   (_size,City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,31,vila franca do campo,37.7167,-25.4333,25.09,61,0,2.01,PT,1723139165
74,74,ghanzi,-21.5667,21.7833,21.18,15,0,4.35,BW,1723139215
76,76,vadso,70.0744,29.7487,21.09,56,0,1.03,NO,1723139217
83,83,hasaki,35.7333,140.8333,26.86,90,0,3.35,JP,1723139225
121,121,lata,40.1629,-8.3327,24.66,52,0,2.92,PT,1723139269


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
31,vila franca do campo,PT,37.7167,-25.4333,61,
74,ghanzi,BW,-21.5667,21.7833,15,
76,vadso,NO,70.0744,29.7487,56,
83,hasaki,JP,35.7333,140.8333,90,
121,lata,PT,40.1629,-8.3327,52,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters or 10 km
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
ghanzi - nearest hotel: Ghanzi farmhouse
vadso - nearest hotel: Scandic Vadsø
hasaki - nearest hotel: Choshi Plaza Hotel
lata - nearest hotel: Residencial Botânico
constantia - nearest hotel: Hotel Dali
alta - nearest hotel: Scandic Alta
koungou - nearest hotel: Hôtel Trévani
prado - nearest hotel: Pousada Recanto do Prado
maun - nearest hotel: Center Lodge Conference Center
tromso - nearest hotel: Clarion Collection Hotel Aurora
los osos - nearest hotel: Back Bay Inn
qingdao - nearest hotel: Le Méridien Qingdao
colonia - nearest hotel: Wasserturm Hotel Cologne
karaisali - nearest hotel: No hotel found
iwaki - nearest hotel: Iwaki Prince Hotel
adler - nearest hotel: Shine
maintirano - nearest hotel: Hotel Safari
aksu - nearest hotel: Akesu Tianyuan International Hotel
lipsko - nearest hotel: INNSIDE by Meliá
shimoda - nearest hotel: ホテルウラガ
xunchang - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
31,vila franca do campo,PT,37.7167,-25.4333,61,Vinha d'Areia Beach Hotel
74,ghanzi,BW,-21.5667,21.7833,15,Ghanzi farmhouse
76,vadso,NO,70.0744,29.7487,56,Scandic Vadsø
83,hasaki,JP,35.7333,140.8333,90,Choshi Plaza Hotel
121,lata,PT,40.1629,-8.3327,52,Residencial Botânico


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [48]:
import hvplot.pandas
!pip install folium
import folium

In [50]:
map_center = [hotel_df['Lat'].mean(), hotel_df['Lng'].mean()]

map_ = folium.Map(location=map_center, zoom_start=2)
for index, row in hotel_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Lng']],
        popup=(
            f"Hotel Name: {row['Hotel Name']}<br>"
            f"City: {row['City']}<br>"
            f"Country: {row['Country']}<br>"
            f"Humidity: {row['Humidity']}%"
        ),
    ).add_to(map_)
    
    display (map_)

# I was unable to plot the map using hvplot (I am working on the issue but I have to submit the assignment in time)
# I decided to look up other ways to do the mapping on the internet and found Folium as one of the solutions. I am
# unsure if the mapping has been done correctly, but I will look into the assignment at a different time to make
# corrections on my own time. For now, due to time constraits, this is what I was able to achieve. 